In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [2]:
def parse_url(url):
    r = requests.get(url)
    if not r.status_code == requests.codes.ok:
        r.raise_for_status()
    r.encoding = r.apparent_encoding
    bs = BeautifulSoup(r.text, 'html.parser')
    return bs

In [3]:
def parse_file(file):
    with open(file) as f:
        bs = BeautifulSoup(f, 'html.parser')
    return bs

In [4]:
def strip_generator(generator):
    list_ = []
    for v in generator:
        if isinstance(v, str):
            list_.append(v.strip())
    return list_

# スポットURL一覧を取得

In [5]:
url = 'https://www.jalan.net/kankou/pro_003/g2_U2/'
bs = parse_url(url)

In [6]:
# path = '../locals/jalan.html'
# bs = parse_file(path)

In [7]:
items = bs.findAll('div', class_='item-listContents')
items

[<div class="item-listContents">
 <div class="item-photo">
 <div class="item-mainImg"><img alt="ねねさんの吹割の滝の投稿写真1" height="171" src="//cdn.jalan.jp/jalan/img/3/kuchikomi/0173/KM/39900_0000173243.jpg" width="228"/></div>
 <ul class="item-thumbs">
 <li><img alt="ベベッチさんの吹割の滝の投稿写真1" height="56" src="//cdn.jalan.jp/jalan/img/3/kuchikomi/0333/KXS/fd222_0000333455.jpg" width="74"/></li>
 <li><img alt="seanさんの吹割の滝の投稿写真1" height="56" src="//cdn.jalan.jp/jalan/img/5/kuchikomi/0005/KXS/2f0e6_0000005063.jpg" width="74"/></li>
 <li><img alt="まみぃままさんの吹割の滝の投稿写真1" height="56" src="//cdn.jalan.jp/jalan/img/0/kuchikomi/0520/KXS/8a45c_0000520529_1.jpg" width="74"/></li>
 </ul>
 </div>
 <div class="item-info">
 <div class="item-innerBtnArea">
 <ul>
 <li class="btnWannagoSpotGray">
 <a data-bookmarktype="2" data-editkbn="1" data-odktype="1" data-page="1" data-spoteventid="guide000000151053" onclick="javascript:doWannagoWent($(this));return false;">行きたい</a>
 </li>
 <li class="btnWentSpotGray">
 <a data-bookma

In [8]:
urls = ['https:' + item.findAll('a')[2]['href'] for item in items]
urls

['https://www.jalan.net/kankou/spt_guide000000151053/',
 'https://www.jalan.net/kankou/spt_guide000000184651/',
 'https://www.jalan.net/kankou/spt_guide000000150978/',
 'https://www.jalan.net/kankou/spt_guide000000164423/',
 'https://www.jalan.net/kankou/spt_guide000000202105/',
 'https://www.jalan.net/kankou/spt_guide000000185267/',
 'https://www.jalan.net/kankou/spt_guide000000195909/',
 'https://www.jalan.net/kankou/spt_guide000000193354/',
 'https://www.jalan.net/kankou/spt_guide000000193353/',
 'https://www.jalan.net/kankou/spt_guide000000192183/',
 'https://www.jalan.net/kankou/spt_guide000000186613/',
 'https://www.jalan.net/kankou/spt_guide000000175433/',
 'https://www.jalan.net/kankou/spt_guide000000164421/']

# スポット詳細ページをパース

In [9]:
url = 'https://www.jalan.net/kankou/spt_guide000000151053/'
bs = parse_url(url)

In [10]:
# path = '../locals/jalan_detail.html'
# bs = parse_file(path)

In [11]:
name = bs.find('h1', class_='detailTitle').string
name

'吹割の滝'

In [12]:
gen = bs.find('div', id='aboutArea').find('p').strings
description = '\n'.join(strip_generator(gen))
print(description)

1936年に天然記念物及び名勝に指定された、東洋のナイアガラとも呼ばれる幅30ｍ高さ７ｍの「吹割の滝」。
群馬県沼田市にあり、利根川水系。
滝から吹き上がる水しぶきが河床を割くように流れているように見えたことで、この名がつけられました。
また、「竜宮の椀」の伝説がある滝で、滝壷は竜宮に通じているとも言われています。
その昔、村人が祝儀の度に、お椀やお膳を借りたいとお願いをした手紙を滝壺に投げ入れたといわれたが、
一度だけ借りたものを返し忘れてしまったことで、それ以来は二度とお椀やお膳を借りれなくなったといわれています。
※天候により遊歩道が通れない場合もあるので要注意。


In [13]:
map = bs.find('div', id='detailMap-canvas')
lat, lon = map['data-lat'], map['data-lng']
print(lat, lon)

36.7023529 139.20641


In [14]:
gs = bs.find('dl', class_='c-genre').findAll('div', class_='dropdownCurrent')[1].find('a').string
gs

'滝・渓谷'

In [15]:
image = 'http:%s' % bs.find('div', class_='detailGallery_box galleryArea-image').find('img')['src']
image

'http://cdn.jalan.jp/jalan/img/3/kuchikomi/0173/KL/39900_0000173243.jpg'

In [16]:
access_text = bs.find('table', class_='basicInfoTable').findAll('tr')[1].find('td').string.strip()
access_text

'(1)関越道沼田ICよりR120経由、日光方面へ30分'

In [20]:
url = 'https://www.jalan.net/kankou/pro_003/g1_22/'
urls = []
pager = [url] + ['%spage_%d/' % (url, i) for i in range(2, 6)]
for page in pager:
    try:
        bs = parse_url(page)
        items = bs.findAll('div', class_='item-listContents')
        urls.extend(['https:' + i.findAll('a')[2]['href'] for i in items])
        sleep(5)
    except requests.exceptions.HTTPError as e:
        break
urls

['https://www.jalan.net/kankou/pro_003/g1_22/', 'https://www.jalan.net/kankou/pro_003/g1_22/page_2/', 'https://www.jalan.net/kankou/pro_003/g1_22/page_3/', 'https://www.jalan.net/kankou/pro_003/g1_22/page_4/', 'https://www.jalan.net/kankou/pro_003/g1_22/page_5/']


['https://www.jalan.net/kankou/spt_14205ab2050129880/',
 'https://www.jalan.net/kankou/spt_08364ab2040005938/',
 'https://www.jalan.net/kankou/spt_13110ab2042041292/',
 'https://www.jalan.net/kankou/spt_09206ab2040006069/',
 'https://www.jalan.net/kankou/spt_12226ab2010146434/',
 'https://www.jalan.net/kankou/spt_08201ab2030005124/',
 'https://www.jalan.net/kankou/spt_13101aj2200023392/',
 'https://www.jalan.net/kankou/spt_08220ab2010003284/',
 'https://www.jalan.net/kankou/spt_13106ab2030004433/',
 'https://www.jalan.net/kankou/spt_guide000000187656/',
 'https://www.jalan.net/kankou/spt_12465ab2050034231/',
 'https://www.jalan.net/kankou/spt_guide000000179765/',
 'https://www.jalan.net/kankou/spt_09202cc3310040305/',
 'https://www.jalan.net/kankou/spt_08201cb3410089210/',
 'https://www.jalan.net/kankou/spt_09206ab2030004392/',
 'https://www.jalan.net/kankou/spt_09383ab2040006204/',
 'https://www.jalan.net/kankou/spt_14214ab2010002020/',
 'https://www.jalan.net/kankou/spt_11363ab201011

# ジャンル一覧を取得

In [5]:
url = 'https://www.jalan.net/kankou/130000/'
s = parse_url(url)

In [14]:
genre_middle = s.find('dl', class_='c-genre').findAll('li')
genre_middle

[<li><a href="//www.jalan.net/kankou/130000/g1_A1/" onclick="javascript:ouComLink('/kankou/130000/g1_A1/','ouw3801Form');$(this).unbind();return false;">アウトドア</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g1_A2/" onclick="javascript:ouComLink('/kankou/130000/g1_A2/','ouw3801Form');$(this).unbind();return false;">ウォータースポーツ・マリンスポーツ</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g1_A3/" onclick="javascript:ouComLink('/kankou/130000/g1_A3/','ouw3801Form');$(this).unbind();return false;">雪・スノースポーツ</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g1_04/" onclick="javascript:ouComLink('/kankou/130000/g1_04/','ouw3801Form');$(this).unbind();return false;">その他スポーツ・フィットネス</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g1_A5/" onclick="javascript:ouComLink('/kankou/130000/g1_A5/','ouw3801Form');$(this).unbind();return false;">エンタメ・アミューズメント</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g1_06/" onclick="javascript:ouComLink('/kankou/130000/g1_06/','ouw3801Form');$

In [15]:
genre_middle_labels = [v.find('a').text for v in genre_middle]
genre_middle_labels

['アウトドア',
 'ウォータースポーツ・マリンスポーツ',
 '雪・スノースポーツ',
 'その他スポーツ・フィットネス',
 'エンタメ・アミューズメント',
 'レジャー・体験',
 'クラフト・工芸',
 '果物・野菜狩り',
 'ミュージアム・ギャラリー',
 '神社・神宮・寺院',
 '伝統文化・日本文化',
 '自然景観・絶景',
 '乗り物',
 '動・植物',
 '風呂・スパ・サロン',
 'ショッピング',
 '観光施設・名所巡り',
 '祭り・イベント',
 '宿泊施設',
 '和食',
 '洋食',
 'イタリアン・フレンチ',
 'アジアン・アジア料理',
 'バー・カクテル',
 'カフェ・スイーツ',
 'その他']

In [16]:
genre_middle_links = [v.find('a').get('href') for v in genre_middle]
genre_middle_links

['//www.jalan.net/kankou/130000/g1_A1/',
 '//www.jalan.net/kankou/130000/g1_A2/',
 '//www.jalan.net/kankou/130000/g1_A3/',
 '//www.jalan.net/kankou/130000/g1_04/',
 '//www.jalan.net/kankou/130000/g1_A5/',
 '//www.jalan.net/kankou/130000/g1_06/',
 '//www.jalan.net/kankou/130000/g1_A7/',
 '//www.jalan.net/kankou/130000/g1_A8/',
 '//www.jalan.net/kankou/130000/g1_A9/',
 '//www.jalan.net/kankou/130000/g1_20/',
 '//www.jalan.net/kankou/130000/g1_12/',
 '//www.jalan.net/kankou/130000/g1_22/',
 '//www.jalan.net/kankou/130000/g1_02/',
 '//www.jalan.net/kankou/130000/g1_09/',
 '//www.jalan.net/kankou/130000/g1_18/',
 '//www.jalan.net/kankou/130000/g1_B1/',
 '//www.jalan.net/kankou/130000/g1_13/',
 '//www.jalan.net/kankou/130000/g1_21/',
 '//www.jalan.net/kankou/130000/g1_05/',
 '//www.jalan.net/kankou/130000/g1_3G004/',
 '//www.jalan.net/kankou/130000/g1_3G005/',
 '//www.jalan.net/kankou/130000/g1_3G006/',
 '//www.jalan.net/kankou/130000/g1_3G009/',
 '//www.jalan.net/kankou/130000/g1_3G012/',
 

In [17]:
url = 'https:' + genre_middle_links[0]
s = parse_url(url)

In [19]:
genre_small = s.find('dl', class_='c-genre').findAll('div', class_='dropdownList')[1].findAll('li')
genre_small

[<li><a href="//www.jalan.net/kankou/130000/g2_N2/" onclick="javascript:ouComLink('/kankou/130000/g2_N2/','ouw3801Form');$(this).unbind();return false;">BBQ(バーベキュー)</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g2_M6/" onclick="javascript:ouComLink('/kankou/130000/g2_M6/','ouw3801Form');$(this).unbind();return false;">トレッキング・登山</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g2_15/" onclick="javascript:ouComLink('/kankou/130000/g2_15/','ouw3801Form');$(this).unbind();return false;">ウォーキング・ハイキング</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g2_N4/" onclick="javascript:ouComLink('/kankou/130000/g2_N4/','ouw3801Form');$(this).unbind();return false;">洞窟体験・ケイビング</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g2_04/" onclick="javascript:ouComLink('/kankou/130000/g2_04/','ouw3801Form');$(this).unbind();return false;">キャンプ場・バンガロー・コテージ等</a></li>,
 <li><a href="//www.jalan.net/kankou/130000/g2_17/" onclick="javascript:ouComLink('/kankou/130000/g2_17/','ouw3801Form');

In [20]:
genre_small_labels = [v.find('a').text for v in genre_small]
genre_small_labels

['BBQ(バーベキュー)',
 'トレッキング・登山',
 'ウォーキング・ハイキング',
 '洞窟体験・ケイビング',
 'キャンプ場・バンガロー・コテージ等',
 'アスレチック(フィールドアスレチック等)',
 '野外レクリエーション',
 'ナイトツアー',
 'オリエンテーリング・パーマネントコース',
 'その他アウトドア',
 'ウォーキングコース']

In [24]:
import time
genre_tree = []
url = 'https://www.jalan.net/kankou/130000/'
s = parse_url(url)
genre_middle = s.find('dl', class_='c-genre').findAll('li')
for g in genre_middle:
    genre_middle_label = g.find('a').text
    url = 'https:' + g.find('a').get('href')
    s = parse_url(url)
    genre_small = s.find('dl', class_='c-genre').findAll('div', class_='dropdownList')[1].findAll('li')
    genre_small_labels = [v.find('a').text for v in genre_small]
    genre_tree.append((genre_middle_label, genre_small_labels))
    time.sleep(5)
genre_tree

['BBQ(バーベキュー)', 'トレッキング・登山', 'ウォーキング・ハイキング', '洞窟体験・ケイビング', 'キャンプ場・バンガロー・コテージ等', 'アスレチック(フィールドアスレチック等)', '野外レクリエーション', 'ナイトツアー', 'オリエンテーリング・パーマネントコース', 'その他アウトドア', 'ウォーキングコース'] ['アウトドア', 'ウォータースポーツ・マリンスポーツ', '雪・スノースポーツ', 'その他スポーツ・フィットネス', 'エンタメ・アミューズメント', 'レジャー・体験', 'クラフト・工芸', '果物・野菜狩り', 'ミュージアム・ギャラリー', '神社・神宮・寺院', '伝統文化・日本文化', '自然景観・絶景', '乗り物', '動・植物', '風呂・スパ・サロン', 'ショッピング', '観光施設・名所巡り', '祭り・イベント', '宿泊施設', '和食', '洋食', 'イタリアン・フレンチ', 'アジアン・アジア料理', 'バー・カクテル', 'カフェ・スイーツ', 'その他']
['スキューバダイビング', 'シュノーケリング・ボートシュノーケル', 'カヌー・カヤック', 'ラフティング', '川下り・ライン下り', 'サップ・SUP(スタンドアップパドル)', '沢下り(キャニオニング)', 'ホバーボード・フライボード', '沢登り(シャワークライミング)', 'サーフィン・ボディボード', 'ウェイクボード・ウェイクサーフィン', 'リバーブギ・ハイドロスピード', 'プール', '素潜り・スキンダイビング', 'その他ウォータースポーツ・マリンスポーツ'] ['アウトドア', 'ウォータースポーツ・マリンスポーツ', '雪・スノースポーツ', 'その他スポーツ・フィットネス', 'エンタメ・アミューズメント', 'レジャー・体験', 'クラフト・工芸', '果物・野菜狩り', 'ミュージアム・ギャラリー', '神社・神宮・寺院', '伝統文化・日本文化', '自然景観・絶景', '乗り物', '動・植物', '風呂・スパ・サロン', 'ショッピング', '観光施設・名所巡り', '祭り・イベント', '宿泊施設', '和食', '洋食', 'イタリアン・フレンチ', 'アジアン・アジア料理

['海鮮', '沖縄料理', '日本料理・懐石', 'ラーメン', '居酒屋'] ['アウトドア', 'ウォータースポーツ・マリンスポーツ', '雪・スノースポーツ', 'その他スポーツ・フィットネス', 'エンタメ・アミューズメント', 'レジャー・体験', 'クラフト・工芸', '果物・野菜狩り', 'ミュージアム・ギャラリー', '神社・神宮・寺院', '伝統文化・日本文化', '自然景観・絶景', '乗り物', '動・植物', '風呂・スパ・サロン', 'ショッピング', '観光施設・名所巡り', '祭り・イベント', '宿泊施設', '和食', '洋食', 'イタリアン・フレンチ', 'アジアン・アジア料理', 'バー・カクテル', 'カフェ・スイーツ', 'その他']
['ステーキ・ハンバーグ・カレー'] ['アウトドア', 'ウォータースポーツ・マリンスポーツ', '雪・スノースポーツ', 'その他スポーツ・フィットネス', 'エンタメ・アミューズメント', 'レジャー・体験', 'クラフト・工芸', '果物・野菜狩り', 'ミュージアム・ギャラリー', '神社・神宮・寺院', '伝統文化・日本文化', '自然景観・絶景', '乗り物', '動・植物', '風呂・スパ・サロン', 'ショッピング', '観光施設・名所巡り', '祭り・イベント', '宿泊施設', '和食', '洋食', 'イタリアン・フレンチ', 'アジアン・アジア料理', 'バー・カクテル', 'カフェ・スイーツ', 'その他']
['フレンチ・フランス料理', 'イタリアン・イタリア料理'] ['アウトドア', 'ウォータースポーツ・マリンスポーツ', '雪・スノースポーツ', 'その他スポーツ・フィットネス', 'エンタメ・アミューズメント', 'レジャー・体験', 'クラフト・工芸', '果物・野菜狩り', 'ミュージアム・ギャラリー', '神社・神宮・寺院', '伝統文化・日本文化', '自然景観・絶景', '乗り物', '動・植物', '風呂・スパ・サロン', 'ショッピング', '観光施設・名所巡り', '祭り・イベント', '宿泊施設', '和食', '洋食', 'イタリアン・フレンチ', 'アジアン・アジア料理', 'バー・カクテル', 'カフェ・スイーツ', 'その

In [26]:
i = 1
list_ = []
for v in genre_tree:
    genre_middle = v[0]
    for genre_small in v[1]:
        list_.append("({0}, '{1}', '{2}')".format(i, genre_small, genre_middle))
        i += 1
print("INSERT INTO jalan_genre_small (jalan_genre_id, genre_small, genre_large) VALUES " + \
', '.join(list_) + ';')

INSERT INTO jalan_genre_small (jalan_genre_id, genre_small, genre_large) VALUES (1, 'BBQ(バーベキュー)', 'アウトドア'), (2, 'トレッキング・登山', 'アウトドア'), (3, 'ウォーキング・ハイキング', 'アウトドア'), (4, '洞窟体験・ケイビング', 'アウトドア'), (5, 'キャンプ場・バンガロー・コテージ等', 'アウトドア'), (6, 'アスレチック(フィールドアスレチック等)', 'アウトドア'), (7, '野外レクリエーション', 'アウトドア'), (8, 'ナイトツアー', 'アウトドア'), (9, 'オリエンテーリング・パーマネントコース', 'アウトドア'), (10, 'その他アウトドア', 'アウトドア'), (11, 'ウォーキングコース', 'アウトドア'), (12, 'スキューバダイビング', 'ウォータースポーツ・マリンスポーツ'), (13, 'シュノーケリング・ボートシュノーケル', 'ウォータースポーツ・マリンスポーツ'), (14, 'カヌー・カヤック', 'ウォータースポーツ・マリンスポーツ'), (15, 'ラフティング', 'ウォータースポーツ・マリンスポーツ'), (16, '川下り・ライン下り', 'ウォータースポーツ・マリンスポーツ'), (17, 'サップ・SUP(スタンドアップパドル)', 'ウォータースポーツ・マリンスポーツ'), (18, '沢下り(キャニオニング)', 'ウォータースポーツ・マリンスポーツ'), (19, 'ホバーボード・フライボード', 'ウォータースポーツ・マリンスポーツ'), (20, '沢登り(シャワークライミング)', 'ウォータースポーツ・マリンスポーツ'), (21, 'サーフィン・ボディボード', 'ウォータースポーツ・マリンスポーツ'), (22, 'ウェイクボード・ウェイクサーフィン', 'ウォータースポーツ・マリンスポーツ'), (23, 'リバーブギ・ハイドロスピード', 'ウォータースポーツ・マリンスポーツ'), (24, 'プール', 'ウォータースポーツ・マリンスポーツ'), (25, '素潜り・スキンダイビング', 'ウォータースポー